In [ ]:
import requests
import time

# Base URL and credentials
QOGITA_API_URL = "https://api.qogita.com"
QOGITA_EMAIL = "jacek.budner@gmail.com"
QOGITA_PASSWORD = "JB100noga!"

# Step 1: Authentication
auth_response = requests.post(
    f"{QOGITA_API_URL}/auth/login/",
    json={"email": QOGITA_EMAIL, "password": QOGITA_PASSWORD}
).json()

if "accessToken" in auth_response:
    access_token = auth_response["accessToken"]
    headers = {"Authorization": f"Bearer {access_token}"}
    cart_qid = auth_response["user"]["activeCartQid"]

    print("Login successful.")
    print(f"Active Cart QID: {cart_qid}\n")

    # Step 2: Paginate through all results
    page = 1
    size = 5000  # Reasonable size per page
    total_count = 0

    while True:
        print(f"Fetching page {page}...")

        search_url = (
            f"{QOGITA_API_URL}/variants/search/?"
            f"page={page}&size={size}"
        )

        response = requests.get(url=search_url, headers=headers).json()

        results = response.get("results", [])
        if not results:
            print("No more results. Done.")
            break

        for variant in results:
            print(f"{variant['gtin']} | {variant['name']} | {variant['categoryName']} | {variant['brandName']} | "
                  f"{variant['price']} | {variant['inventory']} | {variant['imageUrl']} ")
            total_count += 1

        page += 1
        time.sleep(0.5)  # Sleep to avoid rate limits

    print(f"\n✅ Total products retrieved: {total_count}")

else:
    print("Login failed:", auth_response)


In [ ]:
print(variant)

## VArianst add to cart 

In [ ]:
import requests

# Credentials
QOGITA_API_URL = "https://api.qogita.com"
QOGITA_EMAIL = "jacek.budner@gmail.com"
QOGITA_PASSWORD = "JB100noga!"

# Session variables
access_token = None
headers = {}
cart_qid = None

def login():
    global access_token, headers, cart_qid
    print("🔐 Logging in...")
    auth_response = requests.post(
        f"{QOGITA_API_URL}/auth/login/",
        json={"email": QOGITA_EMAIL, "password": QOGITA_PASSWORD}
    ).json()

    if "accessToken" not in auth_response:
        print("❌ Login failed:", auth_response)
        exit()

    access_token = auth_response["accessToken"]
    headers = {"Authorization": f"Bearer {access_token}"}
    cart_qid = auth_response["user"]["activeCartQid"]
    print("✅ Authenticated successfully.")
    print(f"🛒 Active Cart QID: {cart_qid}")

def safe_request(method, url, **kwargs):
    """Wrapper to handle 401 errors by re-logging in and retrying once."""
    global headers
    response = requests.request(method, url, headers=headers, **kwargs)
    if response.status_code == 401:
        print("🔁 Token expired, re-authenticating...")
        login()
        response = requests.request(method, url, headers=headers, **kwargs)
    return response

# Initial login
login()

# Step 2: Get all variants (page by page)
page = 1
while True:
    search_response = safe_request(
        "GET",
        f"{QOGITA_API_URL}/variants/search/?page={page}&size=1000"
    ).json()

    results = search_response.get("results", [])
    if not results:
        print("❌ No more products found.")
        break

    for variant in results:
        fid = variant["fid"]
        slug = variant["slug"]
        variant_name = variant["name"]

        offers_url = f"{QOGITA_API_URL}/variants/{fid}/{slug}/offers/"
        offers_raw_response = safe_request("GET", offers_url)

        if not offers_raw_response.ok:
            print(f"❌ Failed to fetch offers for {variant_name} | Status Code: {offers_raw_response.status_code}")
            print("🔎 Response text:", offers_raw_response.text)
            continue

        try:
            offers_response = offers_raw_response.json()
        except requests.exceptions.JSONDecodeError:
            print(f"❌ Could not decode JSON for offers of {variant_name}")
            print("🔎 Raw response content:", offers_raw_response.text)
            continue

        offers = offers_response.get("offers", []) if isinstance(offers_response, dict) else []

        if not offers:
            print(f"❌ No offers found for {variant_name}.")
            continue

        quantity = 100

        valid_offers = [
            offer for offer in offers
            if quantity * float(offer["price"]) >= float(offer["mov"])
        ]

        if not valid_offers:
            print(f"⚠️ No valid offers for {variant_name} meet the MOV requirement.")
            continue

        best_offer = min(valid_offers, key=lambda x: float(x["price"]))
        offer_qid = best_offer["qid"]
        total_price = float(best_offer["price"]) * quantity

        print(f"📦 Selected Offer for {variant_name}:")
        print(f"    Seller: {best_offer['seller']}")
        print(f"    Price: €{best_offer['price']}")
        print(f"    MOV: €{best_offer['mov']}")
        print(f"    Total Order Value: €{total_price:.2f}")

        # Add to cart
        add_to_cart_response = safe_request(
            "POST",
            f"{QOGITA_API_URL}/carts/{cart_qid}/lines/",
            json={"offerQid": offer_qid, "quantity": quantity}
        )

        if add_to_cart_response.ok:
            print(f"✅ Added {quantity} units of {variant_name} to cart.\n")
        else:
            try:
                print(f"❌ Failed to add {variant_name} to cart:", add_to_cart_response.json())
            except:
                print(f"❌ Failed to add {variant_name} to cart. Raw response:", add_to_cart_response.text)

    page += 1


In [ ]:
import requests
import pandas as pd
import os

# Credentials
QOGITA_API_URL = "https://api.qogita.com"
QOGITA_EMAIL = "jacek.budner@gmail.com"
QOGITA_PASSWORD = "JB100noga!"

# Step 1: Login
auth_response = requests.post(
    f"{QOGITA_API_URL}/auth/login/",
    json={"email": QOGITA_EMAIL, "password": QOGITA_PASSWORD}
).json()

if "accessToken" not in auth_response:
    print("Login failed:", auth_response)
    exit()

access_token = auth_response["accessToken"]
headers = {"Authorization": f"Bearer {access_token}"}
cart_qid = auth_response["user"]["activeCartQid"]

print("✅ Authenticated successfully.")
print(f"🛒 Active Cart QID: {cart_qid}")

# Step 2: Search for product variant
query = "Yara by Lattafa Perfumes Eau De Parfum 100ml 3.4 fl oz for Women"
search_response = requests.get(
    f"{QOGITA_API_URL}/variants/search/?query={query}&page=1&size=10",
    headers=headers
).json()

results = search_response.get("results", [])
if not results:
    print("❌ No products found.")
    exit()

variant = results[0]
fid = variant["fid"]
slug = variant["slug"]
product_name = variant["name"]

# Step 3: Get offers from specific variant
offers_url = f"{QOGITA_API_URL}/variants/{fid}/{slug}/offers/"
offers_response = requests.get(offers_url, headers=headers).json()
print(offers_response)

# Extract offers list
offers = offers_response.get("results", []) if isinstance(offers_response, dict) else offers_response

if not offers:
    print("❌ No offers found.")
    exit()

# Build a clean structured list of offers
product_offers = []
for offer in offers:
    offer["product"] = product_name  # Attach product name to each offer
    product_offers.append(offer)

# Create DataFrame and select useful columns
df = pd.DataFrame(product_offers)

# Optional: rearrange or select specific columns 
desired_columns = ['product', 'qid', 'seller', 'price', 'priceCurrency', 'inventory', 'mov']
df = df[[col for col in desired_columns if col in df.columns]]

# Save to CSV
csv_file = "qogita_offers.csv"
if not os.path.exists(csv_file):
    df.to_csv(csv_file, index=False)
    print(f"💾 Created new CSV file: {csv_file}")
else:
    df.to_csv(csv_file, mode='a', header=False, index=False)
    print(f"📥 Appended offers to: {csv_file}")



✅ Authenticated successfully.
🛒 Active Cart QID: 69eb3544-57f1-4da4-b17d-e72a3c467c4b
{'offers': [{'seller': 'GELD3O', 'price': '13.48', 'priceCurrency': 'EUR', 'inventory': 845, 'mov': '350.00', 'movCurrency': 'EUR', 'movProgress': '0.000', 'quantityInCart': 0, 'unit': 1, 'isTraceable': True, 'qid': '4202783a-a1b3-4944-9e33-b2fef8be0e97'}, {'seller': 'O8V81R', 'price': '13.81', 'priceCurrency': 'EUR', 'inventory': 476, 'mov': '600.00', 'movCurrency': 'EUR', 'movProgress': '0.000', 'quantityInCart': 0, 'unit': 1, 'isTraceable': True, 'qid': 'c474f183-7291-4dc0-8da5-5a86a42999f8'}, {'seller': 'EMJVM7', 'price': '13.92', 'priceCurrency': 'EUR', 'inventory': 960, 'mov': '600.00', 'movCurrency': 'EUR', 'movProgress': '0.000', 'quantityInCart': 0, 'unit': 1, 'isTraceable': True, 'qid': 'fefd11b0-068f-498d-85bd-d99df0bfa43a'}, {'seller': 'LEKM75', 'price': '14.92', 'priceCurrency': 'EUR', 'inventory': 480, 'mov': '350.00', 'movCurrency': 'EUR', 'movProgress': '0.000', 'quantityInCart': 0, 'u

: 

In [4]:
pip install requests pandas

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 77.7 kB/s eta 0:00:0000:0100:05
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 82.4 kB/s eta 0:00:0000:0100:05
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 509.2/509.2 kB 72.0 kB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 347.8/347.8 kB 89.3 kB/s eta 0:00:00a 0:00:01

[notice] A new release of pip available: 22.3 -> 25.0.1
[notice] To update, run: pip3.11 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import requests
import csv

# Credentials
QOGITA_API_URL = "https://api.qogita.com"
QOGITA_EMAIL = "jacek.budner@gmail.com"
QOGITA_PASSWORD = "JB100noga!"

# Session variables
access_token = None
headers = {}
cart_qid = None

# Prepare CSV File (open in append mode so data is added incrementally)
csv_file = open('variants_sellers.csv', mode='w', newline='', encoding='utf-8')
csv_writer = csv.writer(csv_file)
csv_writer.writerow([
    'GTIN', 'Variant Name', 'Category Name', 'Brand Name', 'Price (€)', 'Inventory', 'Image URL', 
    'Seller', 'MOV (€)', 'Available Qty', 'Ordering Qty', 'Total Price (€)',"Unit"
])

def login():
    global access_token, headers, cart_qid
    print("🔐 Logging in...")
    auth_response = requests.post(
        f"{QOGITA_API_URL}/auth/login/",
        json={"email": QOGITA_EMAIL, "password": QOGITA_PASSWORD}
    ).json()

    if "accessToken" not in auth_response:
        print("❌ Login failed:", auth_response)
        exit()

    access_token = auth_response["accessToken"]
    headers = {"Authorization": f"Bearer {access_token}"}
    cart_qid = auth_response["user"]["activeCartQid"]
    print("✅ Authenticated successfully.")
    print(f"🛒 Active Cart QID: {cart_qid}")

def safe_request(method, url, **kwargs):
    """Wrapper to handle 401 errors by re-logging in and retrying once."""
    global headers
    response = requests.request(method, url, headers=headers, **kwargs)
    if response.status_code == 401:
        print("🔁 Token expired, re-authenticating...")
        login()
        response = requests.request(method, url, headers=headers, **kwargs)
    return response

# Initial login
login()

# Step 2: Get all variants (page by page)
page = 1
while True:
    search_response = safe_request(
        "GET",
        f"{QOGITA_API_URL}/variants/search/?page={page}&size=1000"
    ).json()

    results = search_response.get("results", [])
    if not results:
        print("❌ No more products found.")
        break

    for variant in results:
        fid = variant["fid"]
        slug = variant["slug"]
        variant_name = variant["name"]

        # Collect additional information
        gtin = variant.get('gtin', '')
        category_name = variant.get('categoryName', '')
        brand_name = variant.get('brandName', '')
        price = variant.get('price', '')
        inventory = variant.get('inventory', '')
        image_url = variant.get('imageUrl', '')

        offers_url = f"{QOGITA_API_URL}/variants/{fid}/{slug}/offers/"
        offers_raw_response = safe_request("GET", offers_url)

        if not offers_raw_response.ok:
            print(f"❌ Failed to fetch offers for {variant_name} | Status Code: {offers_raw_response.status_code}")
            print("🔎 Response text:", offers_raw_response.text)
            continue

        try:
            offers_response = offers_raw_response.json()
        except requests.exceptions.JSONDecodeError:
            print(f"❌ Could not decode JSON for offers of {variant_name}")
            print("🔎 Raw response content:", offers_raw_response.text)
            continue

        offers = offers_response.get("offers", []) if isinstance(offers_response, dict) else []

        if not offers:
            print(f"❌ No offers found for {variant_name}.")
            continue

        # Start with the requested quantity
        requested_quantity = 100

        # Find valid offers that meet the MOV requirement
        valid_offers = [
            offer for offer in offers
            if requested_quantity * float(offer["price"]) >= float(offer["mov"])
        ]

        if not valid_offers:
            print(f"⚠️ No valid offers for {variant_name} meet the MOV requirement.")
            continue

        # Choose the best offer (min price)
        best_offer = min(valid_offers, key=lambda x: float(x["price"]))
        offer_qid = best_offer["qid"]

        # Check available quantity and adjust order quantity if needed
        available_quantity = best_offer.get("availableQuantity", 0)
        quantity_to_order = min(requested_quantity, available_quantity)
        total_price = float(best_offer["price"]) * quantity_to_order

        # Save to CSV immediately before printing
        csv_writer.writerow([
            gtin, variant_name, category_name, brand_name, price, inventory, image_url, 
            best_offer['seller'], best_offer['mov'], available_quantity, quantity_to_order, f"{total_price:.2f}",best_offer["unit"]
        ])
        csv_file.flush()  # Ensures data is written to the file immediately

        # Print the output after saving the data
        print(f"📦 Selected Offer for {variant_name}:")
        print(f"    GTIN: {gtin}")
        print(f"    Category: {category_name}")
        print(f"    Brand: {brand_name}")
        print(f"    Price: €{price}")
        print(f"    Inventory: {inventory}")
        print(f"    Image URL: {image_url}")
        print(f"    Seller: {best_offer['seller']}")
        print(f"    MOV: €{best_offer['mov']}")
        print(f"    Available: {available_quantity}")
        print(f"    Ordering: {quantity_to_order} units | Total: €{total_price:.2f}")
        print(f"unit is : {best_offer["unit"]}")

        # Add to cart
        add_to_cart_response = safe_request(
            "POST",
            f"{QOGITA_API_URL}/carts/{cart_qid}/lines/",
            json={"offerQid": offer_qid, "quantity": quantity_to_order}
        )

        if add_to_cart_response.ok:
            print(f"✅ Added {quantity_to_order} units of {variant_name} to cart.\n")
        else:
            try:
                print(f"❌ Failed to add {variant_name} to cart:", add_to_cart_response.json())
            except:
                print(f"❌ Failed to add {variant_name} to cart. Raw response:", add_to_cart_response.text)

    page += 1

# Close the CSV file after processing all pages
csv_file.close()
print("💾 Data saved to variants_sellers.csv")


🔐 Logging in...
✅ Authenticated successfully.
🛒 Active Cart QID: 69eb3544-57f1-4da4-b17d-e72a3c467c4b
📦 Selected Offer for Yara by Lattafa Perfumes Eau De Parfum 100ml 3.4 fl oz for Women:
    GTIN: 6291108730515
    Category: Eau De Parfum
    Brand: Lattafa
    Price: €13.82
    Inventory: 456
    Image URL: https://static.prod.qogita.com/files/images/variants/TWttPXFk3XuSBGeaRmb2jj.jpg
    Seller: O8V81R
    MOV: €600.00
    Available: 0
    Ordering: 0 units | Total: €0.00
❌ Failed to add Yara by Lattafa Perfumes Eau De Parfum 100ml 3.4 fl oz for Women to cart: {'code': 'invalid', 'message': 'Invalid input.', 'quantity': ['Ensure this value is greater than or equal to 1.']}
📦 Selected Offer for Afnan 9 PM Eau De Parfum 100ml:
    GTIN: 6290171002338
    Category: Eau De Parfum
    Brand: Afnan
    Price: €17.47
    Inventory: 500
    Image URL: https://static.prod.qogita.com/files/images/variants/VhLdYFBqgAFfk8ps9Jitu8.jpg
    Seller: 16E6GQ
    MOV: €350.00
    Available: 0
    Or

KeyboardInterrupt: 

In [2]:
print(best_offer)

{'seller': 'GELD3O', 'price': '52.99', 'priceCurrency': 'EUR', 'inventory': 115, 'mov': '350.00', 'movCurrency': 'EUR', 'movProgress': '0.000', 'quantityInCart': 0, 'unit': 1, 'isTraceable': True, 'qid': '4dae8d1d-17e4-48c9-b27b-3a873e81e4dd'}
